<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/1.1_CMV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install netCDF4

In [ ]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import io, os, sys, setuptools, tokenize

In [ ]:
## import modules

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from datetime import datetime
from datetime import timedelta
from prophet import Prophet

from scipy.fft import fft

In [ ]:
latitude = 50.2
longitude = 7.8